In [1]:
import requests, math, re, sys, time, json
from bs4 import  BeautifulSoup as bs

In [2]:
rs = requests.session()
res = rs.get('http://vacation.eztravel.com.tw/pkgfrn/results/TPE/49?fullStatus=NONE')
soup = bs(res.text, "lxml")

## 抓取頁數 & 抓取連結

In [3]:
count  = soup.select('.v-line')[0].text.split(' ') #抓出有幾筆資料
print count[1]
page = int(math.ceil(float(count[1]) / 18)) #算出總頁數
print page

117
7


In [4]:
urlFormat = 'http://vacation.eztravel.com.tw/pkgfrn/results/TPE/49/{}?&fullStatus=NONE' #建立換頁的網址字串
with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','w') as f:
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
    for pageNum in range(1, page + 1): #從第一頁到第七頁的前一頁停止，故再多加一
        url = urlFormat.format(pageNum)
        res2 = rs.get(url)
        soup1 = bs(res2.text, "lxml")
        list_table = soup1.select('div#listTable')[0]
        for items in list_table.select('div.list-box.mainLinkBox'):
            f.write('http://vacation.eztravel.com.tw/pkgfrn/' + items.select('a')[0]['href'].split('/pkgfrn/')[1] + '\n')
            #依照內容規則變化，加上字串相加，將網址寫入到txt檔中

## 檢查是否有重複的key與value

In [5]:
str = 'http://vacation.eztravel.com.tw/pkgfrn/introduction/FRN0000014050/20160524'
get = str.split('/')
print get

['http:', '', 'vacation.eztravel.com.tw', 'pkgfrn', 'introduction', 'FRN0000014050', '20160524']


In [5]:
dicCheck = {} #宣告一個dic，稍後會將檔名(Key)跟網址(Value)一一放入
num = 0
for bid_url in open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt','r'): #讀連結檔
    num += 1
    splitStr = bid_url.split('/') #將網址切割，並且丟入splitStr這個List裡
    linkName = splitStr[5].strip() + "_" + splitStr[6].strip() #將List[index]對應到的元素，以字串型式組合成dataNmae
    #用商品編號當作檔名
    if linkName not in dicCheck:
        print 'No.' + str(num) + ', ' + linkName +  ' is new, and added in dicCheck.'
        dicCheck[linkName] = bid_url
    else:
        print 'No.' + str(num) + ', ' + linkName + ' is existed, and not added in dicCheck.'
        print linkName, dicCheck[linkName] #若有重複，則print出重複的Key跟Value

No.1, VDR0000001943_NGO04IT6621H is new, and added in dicCheck.
No.2, FRN0000006441_20160603 is new, and added in dicCheck.
No.3, VDR0000001888_NGO04160614A is new, and added in dicCheck.
No.4, VDR0000001943_NGO04IT6904H is new, and added in dicCheck.
No.5, VDR0000001279_DTS-A0613NN2 is new, and added in dicCheck.
No.6, VDR0000007986_NGO05CI61416C_485 is new, and added in dicCheck.
No.7, VDR0000001846_16JP605CIA is new, and added in dicCheck.
No.8, VDR0000001913_TOM05IT160617I is new, and added in dicCheck.
No.9, VDR0000001943_NGO04IT6703H is new, and added in dicCheck.
No.10, VDR0000001975_KMQ04160927TA is new, and added in dicCheck.
No.11, VDR0000001888_NGO05160601I is new, and added in dicCheck.
No.12, VDR0000001943_NRT05IT6612D is new, and added in dicCheck.
No.13, FRN0000014083_20160622 is new, and added in dicCheck.
No.14, VDR0000001888_NGO05160622A is new, and added in dicCheck.
No.15, VDR0000001914_NGO05160903L is new, and added in dicCheck.
No.16, VDR0000001975_KMQ05160623TA i

## 將需要的資料抓入檔案

In [6]:
### 印出字典的key與value########
def PrintKeyValue(dic_in):
    for key, value in dic_in.iteritems():
        print key, ':', value 

In [12]:
itemDetail = {} #放置所有細項資料集合
eachTravel = [] #放置所有細項資料於旅行方案集合
totalTravel = {} #放置所有的旅行方案於總集合
totalCount = 0
errorCount = 0
with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Shikoku.txt', 'w') as errorFile:
    #寫error檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Shikoku.txt
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\Error\\error_list_for_EZtravel_in_Shikoku.txt
    for bid_url in open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt', 'r'):
        #讀連結檔
        #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
        #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Link\\bid_list_for_EZtravel_in_Shikoku.txt
        try: #使用try-except
            daysHotFormat = 'day{}_Hotel' #建立字串格式
            daysTourFormat = 'day{}_Tour' #同上
            bid_detail = requests.get(bid_url.strip()) #讀取檔案中的連結
            soup2 = bs(bid_detail.text, "lxml") #萃取資料
            row = soup2.select('div#pkgfrnVisitHistory')[0]
            price = re.search('\d+', row.select('.price')[0].text) #利用正規表示法抓取價格
            plane = re.search('.\W{2,4}', row.select('div#flight-block1 span')[3].text) #利用正規表示法抓取航空公司名稱
            daysTransfer = (row.select('div.product-info.css-td')[0].text.strip().split(u'天數')) #抓每筆旅行方案的旅行天數
            days = (re.search('\d{1}', daysTransfer[1])) #利用正規表示法抓出旅行天數
            itemDetail['Title'] = row.select('div.pro-title.css-td h1')[0].text #將個別資料丟入itemDetail字典中，以下相同
            itemDetail['Price'] = price.group(0)
            itemDetail['City'] = row.select('.tag')[0].text.strip()
            itemDetail['Time'] = row.select('div#flight-block1 span')[2].text.strip()
            itemDetail['PlaneCom'] = plane.group(0)

            if len(row.select('div.pro-id-right.css-td span')[0].text.strip()) > 0: #判斷若PID_1長度大於零，則將資料丟入
                itemDetail['PID_1'] = row.select('div.pro-id-right.css-td span')[0].text.strip()
            else: #判斷若PID_1長度小於零，則丟入Null
                itemDetail['PID_1'] = 'Null'
            if len(row.select('div.pro-id-right.css-td span')[1].text.strip()) > 0: #同上
                itemDetail['PID_2'] = row.select('div.pro-id-right.css-td span')[1].text.strip()
            else: #同上
                itemDetail['PID_2'] = 'Null'
            totalCount += 1 #計算有多少筆資料丟入JSON檔中

            print '----------------------------這是分隔線----------------------------'
            print 'days: ' + days.group(0) #印出當前旅行方案的旅行天數
            print 'Url: ' + bid_url.strip() #印出當前旅行方案的網址

            #因為EZtravel的網站資料較鬆散，且只有旅館跟行程有規律性，故特別獨立出來寫加入的方法
            num = 1 #因住宿飯店的索引值是[1]、[3]、[5]……的奇數，所以給起始值，每執行一次會增加2
            for times in range(0,int(days.group(0))):
                daysHotInput = daysHotFormat.format(times + 1) #這邊是天數，day{1}_Hotel、day{2}_Hotel……
                itemDetail[daysHotInput] = row.select('div.meals')[num].text.strip()
                #這邊是索引值(奇數)，row.select('div.meals')[1].text.strip()、row.select('div.meals')[3].text.strip()……
                num += 2
            for indexTour in range(0,int(days.group(0))):
                daysTourInput = daysTourFormat.format(indexTour + 1) #這邊是天數，day{1}_Tour、day{2}_Tour……
                itemDetail[daysTourInput] = row.select('div.day-box h4')[indexTour].text.strip()
                #這邊是索引值，row.select('div.day-box h4')[1].text.strip()、row.select('div.day-box h4')[2].text.strip()……
            PrintKeyValue(itemDetail)
            eachTravel.append(itemDetail)
            totalTravel['eachTravel'] = eachTravel

        except:
            errorCount += 1
            splitPID = bid_url.strip().split('/')
            strPID = splitPID[5].strip() + "_" + splitPID[6].strip()
            #當錯誤訊息出現時，印出是哪一筆資料出現錯誤
            print '----------------' + strPID + ' is error----------------'
                #將出現錯誤的資料網址，寫入txt檔中
            errorFile.write(bid_url)

with open('E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json', 'w') as dataFile:
    #寫最後全部資料檔
    #桌電位置:E:\\BigData\\Project\\Travel\\EZTravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json
    #筆電位置:D:\\Big Data\\Project\\Travel\\EZtravel\\Japan\\Data\\totalTravel_for_EZtravel_in_Shikoku.json
    json.dump(totalTravel, dataFile)

print '----------------------------All datas are done!!!----------------------------'
print 'Data Quantities: ' + str(totalCount)
print 'Error Quantities: ' + str(errorCount)

----------------------------這是分隔線----------------------------
days: 4
Url: http://vacation.eztravel.com.tw/pkgfrn/introduction/VDR0000001943/NGO04IT6621H
City : 名古屋
PID_1 : VDR0000001943
day4_Hotel : 或同級
day4_Tour : 日本海上機場～【名古屋中部國際空港】～機場內大正建築飲食商店街－名古屋(中部國際空港)／台北(桃園國際機場)
PlaneCom : 台灣虎航 
PID_2 : NGO04IT6621H
Price : 19900
Title : 名名立山真好玩～漫步大雪谷(4/16~6/22).上高地.飛驒高山小京都.白川鄉合掌村.溫泉4日
day1_Hotel : 金澤MY STAY、HOTEL金澤、金澤國際 、金澤APA、栗津GRAND、富山APA或同級或同級
day3_Hotel : 中部機場COMFORT、東橫INN、岐阜リソル或同級
day2_Tour : 福井或富山或石川－立山黑部阿爾卑斯山脈路線＜立山黑部(立山纜車)～美女平(高原巴士)～室堂(無軌電車)～大觀峰(空中索道纜車)～黑部平(地下電車)～黑部湖(徒步)～黑部水壩(無軌電車)～扇澤＞－長野或歧阜 ※ 立山雪壁奇景：【雪の大谷】散策預定於4月16日～6月22日期間開放
day2_Hotel : 奧飛驒溫泉 穗高莊、高山GREEN、高山飛驒PLAZA、高山格蘭比亞、白馬GREEN PLAZA或同級
Time : 06:20 - 10:05
day3_Tour : 奧飛驒溫泉－上高地（大正池、河童橋，被日本人視作神的故鄉，秘境首選，宛如人間仙境)－【日本米其林三星觀光景點】高山上三之町古街巡禮(三筋町小京都)－免稅店－名古屋市區自由逛街購物－名古屋(歧阜)
day1_Tour : 台北(桃園國際機場) / 名古屋(中部國際機場) ～郡上八幡古城區巡禮~名水百選宗祇水~郡上博覽館．郡上舞見學～ 白川鄉合掌村(世界文化遺產) ～金澤(或富山或福井栗津溫泉)
----------------------------這是分隔線----------------------------
days: 4


## JSON LINK: http://stackoverflow.com/questions/7100125/storing-python-dictionaries